# 2.2 数据预处理

为了能用深度学习来解决真实世界的问题，需要从现实中获取海量数据，这些原始数据通常需要经过预处理后，才能够转化为torch张量的格式，进行进一步的计算与分析。
在python数据分析工具中，通常使用pandas库，它能很好地与torch张量兼容。本节主要介绍使用pandas预处理原始数据，并将其转换为张量格式的步骤。后面再学习各种数据预处理技术。

## 2.2.1 读取数据集

举个例子，我们先创建一个人工数据集，并存储在.csv文件（../data/house_tiny.csv）中。以其他格式存储的数据也可以通过类似的方法操作。

In [1]:
import os # 导入os库

# 创建文件夹，路径为“./data”，exist_ok=True表示当文件夹已经存在时不会报警，无论是True还是False，都不会覆盖已存在的文件夹。
# 创建路径中./代表当前文件所在的目录，../代表当前文件所在的目录所在的目录，也就是上一级目录
os.makedirs(os.path.join('.', 'data'), exist_ok=True) 

# os.path.join函数返回一个字符串，表示连接后的路径。这里是数据文件的路径，为“./data/house_tiny.csv”
# 使用os.path.join，可以在构建文件路径时不必担心使用操作系统特定的路径分隔符（如/或\）。这样可以编写更具可移植性的代码。
data_file = os.path.join('.', 'data', 'house_tiny.csv')

with open(data_file, 'w') as f:
    # 如果data_file指定的文件不存在，Python会自动创建一个新文件，然后以写入模式打开该文件。
    # 如果data_file指定的文件已经存在，将清空文件中的内容，然后开始写入新的内容。
    
    f.write('NumRooms,Alley,Price\n') #  列名
    f.write('NA,Pave,12750.2\n') # 每一行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要从创建的.csv文件中读取原始的数据集，我们导入pandas包并调用read_csv函数。该数据集有四行三列，每行是一个样本，每列是一个特征或标签，分别是NumRooms、Alley、Price，其中Price是标签。

In [2]:
import pandas as pd
data = pd.read_csv(data_file)
print(data, type(data))

   NumRooms Alley     Price
0       NaN  Pave   12750.2
1       2.0   NaN  106000.0
2       4.0   NaN  178100.0
3       NaN   NaN  140000.0 <class 'pandas.core.frame.DataFrame'>


在pandas中，默认情况下，它会将一些常见的缺失值标识符（如"NA"、"N/A"、"NULL"等）解释为缺失值（NaN，Not a Number）。这是为了方便处理数据时能够统一处理缺失值。

## 2.2.2 处理缺失值 

NAN代表缺失值。处理缺失的数据的典型方法包括插值法和删除法，其中插值法用一个替代值弥补缺失值，如0，删除法则直接忽略缺失值，这里用插值法。

In [3]:
inputs, outputs = data.iloc[:, :-1], data.iloc[:, -1]
outputs

0     12750.2
1    106000.0
2    178100.0
3    140000.0
Name: Price, dtype: float64

pandas类型的表格，可以用iloc属性获取索引中的元素，若直接data[:]不用iloc。这里是将前面的列看作特征，作为inputs；最后一列看作标签，作为outputs。

In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,Alley_Pave,Alley_nan
0,NaN,True,False
1,2.0,False,True
2,4.0,False,True
3,NaN,False,True


对inputs的每一列进行处理，如果含有非数字元素(如Pave)，就对该列进行扩展，假如该列有n种不同的元素，就扩展为n列，也就是n类二元特征，对应的值变为True/False，用于指示原数据中该行的样本的该特征是否为该类。dummy_na=True表示将NAN也看作一类。

In [5]:
inputs[:] = inputs.fillna(inputs.mean()) # 对纯数字列每列取平均，插入该列的缺失值
inputs[:] = inputs.astype(int) # 将bool值转换成0和1
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


## 2.2.3 转换为张量格式 

In [6]:
import torch
# pandas表格的values属性是一个numpy数组，除去了列名和行名
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([ 12750.2000, 106000.0000, 178100.0000, 140000.0000],
        dtype=torch.float64))